# PePR: Direct Generalized Membership Inference Attack
Attack a single target model with a singe attack configuration.

## Prep Google Colab Enviornment
**Important: Restart the Runtime after this Cell!**
The restart is needed because of `pip install ./gmia`.

In [ ]:
!git clone https://github.com/hallojs/ml-pepr.git
%pip install ./ml-pepr
%pip install pylatex

## Imports

In [ ]:
from pepr.privacy import gmia
from pepr.utilities import assign_record_ids_to_target_models

import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense

import numpy as np
import logging

## Setup Logging

In [ ]:
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s', '%Y-%m-%d %H:%M:%S')

# TensorFlow Logger
file_handler_tf = logging.FileHandler('tf.log')
file_handler_tf.setLevel(logging.INFO)
file_handler_tf.setFormatter(formatter)

tf.get_logger().setLevel(logging.INFO)
logger_tf = tf.get_logger()
logger_tf.addHandler(file_handler_tf)

# PePR Logger
file_handler_pr = logging.FileHandler('pepr.privacy.gmia.log')
file_handler_pr.setLevel(logging.DEBUG)
file_handler_pr.setFormatter(formatter)

stream_handler_pr = logging.StreamHandler()
stream_handler_pr.setLevel(logging.DEBUG)
stream_handler_pr.setFormatter(formatter)

logger_pr = logging.getLogger('pepr.privacy.gmia')
logger_pr.addHandler(file_handler_pr)
logger_pr.addHandler(stream_handler_pr)

## A Few Function Definitions

In [ ]:
def create_model(input_shape, n_categories):
  """Architecture of the target and reference models.

  Parameters
  ----------
  input_shape : tuple
      Dimensions of the input for the target/training
  n_categories : int
      number of categories for the prediction
  models.

  Returns
  -------
  tensorflow.python.keras.engine.sequential.Sequential
      A convolutional neuronal network model.
  """
  model = Sequential()

  # first convolution layer
  model.add(Conv2D(filters=32, kernel_size=(5, 5), strides=(
      1, 1), padding='same', input_shape=input_shape))
  model.add(Activation('relu'))

  # max pooling layer
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

  # second convolution layer
  model.add(Conv2D(filters=64, kernel_size=(
      5, 5), strides=(1, 1), padding='same'))
  model.add(Activation('relu'))

  # max pooling layer
  model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

  # fully connected layer
  model.add(Flatten())
  model.add(Dense(1024))
  model.add(Activation('relu'))

  # drop out
  model.add(Dropout(rate=0.5))

  # fully connected layer
  model.add(Dense(n_categories))
  model.add(Activation('softmax'))

  return model

def create_compile_model():
  """Create compiled model.
  
  At the moment pepr.gmia needs this function to train the reference models.

  Returns
  -------
    tensorflow.python.keras.engine.sequential.Sequential
      A compiled tensorflow model.
  """
  input_shape = (28, 28, 1)
  number_classes = 10

  model = create_model(input_shape, number_classes)

  optimizer = optimizers.Adam(lr=0.0001)
  loss = 'categorical_crossentropy'
  metrics = ["accuracy"]
  model.compile(optimizer, loss=loss, metrics=metrics)

  return model

def load_fashion_mnist():
    """Loads and preprocesses the fashion mnist dataset.

    Returns
    -------
    tuple
        (training data, training labels, test data, test labels)
    """
    train, test = tf.keras.datasets.fashion_mnist.load_data()
    train_data, train_labels = train
    test_data, test_labels = test

    # Normalize the data to a range between 0 and 1
    train_data = np.array(train_data, dtype=np.float32) / 255
    test_data = np.array(test_data, dtype=np.float32) / 255

    # Reshape the images to (28, 28, 1)
    train_data = train_data.reshape(train_data.shape[0], 28, 28, 1)
    test_data = test_data.reshape(test_data.shape[0], 28, 28, 1)

    train_labels = np.array(train_labels, dtype=np.int32)
    test_labels = np.array(test_labels, dtype=np.int32)

    return np.vstack((train_data, test_data)), np.hstack((train_labels, test_labels))

## Data Setup

In [ ]:
data, labels = load_fashion_mnist()

## Train a Target Model

In [ ]:
target_model = create_compile_model()
target_model.fit(data[40000:50000],
                 tf.keras.utils.to_categorical(labels[40000:50000], num_classes=10),
                 epochs=50,
                 batch_size=50,
                 verbose=0)
target_model.save('data/target_model')

## Run the Attack

In [ ]:
attack_pars = {
    'number_classes': 10,
    'number_reference_models': 100,
    'reference_training_set_size': 10000,
    'create_compile_model': create_compile_model,
    'reference_epochs': 50,
    'reference_batch_size': 50,
    'hlf_metric': 'cosine',
    'hlf_layer_number': 10,
    'number_target_records': 25
}

# single target
data_conf = {
    'reference_indices': list(range(40000)),
    'target_indices': list(range(40000, 50000)),
    'evaluation_indices': list(range(40000, 60000)),
    'record_indices_per_target': np.array([np.arange(10000)])
}

gmia_attack = gmia.DirectGmia('Playground GMIA', attack_pars, data, labels, data_conf, [target_model])

gmia_attack.run(save_path='data')

gmia_attack.create_attack_report()

In [ ]:
# Zip report directory if you want to download it from google colab
!zip -r gmia_report.zip gmia_report